### 1. Load the Data files

In [ ]:
#importing relevant libraries for reading datasets
import csv
from csv import reader
from xml.dom import minidom
import json

#### 1.1. Load the csv data

In [ ]:
def read_csv_file(file):
    db_list = []
    with open(file, 'r') as csv_file:
        csv_db = csv.DictReader(csv_file)
        for entry in csv_db:
            db_list.append(entry)
    return db_list

In [ ]:
csv_db = read_csv_file('user_data.csv')

#### 1.2. Load the XML data

In [ ]:

def read_xml_file(file):
    data = minidom.parse(file) #parse xml file
    users = data.getElementsByTagName('user') #getting users data from the user tag
    #converting the users data into list of dictionaries
    xml_list = []
    for user in users:
        xml_data = {}
        xml_data['First Name'] = user.attributes['firstName'].value
        xml_data['Last Name'] = user.attributes['lastName'].value
        xml_data['age'] = user.attributes['age'].value
        xml_data['sex'] = user.attributes['sex'].value
        xml_data['retired'] = user.attributes['retired'].value
        xml_data['dependants'] = user.attributes['dependants'].value
        xml_data['marital_status'] = user.attributes['marital_status'].value
        xml_data['salary'] = user.attributes['salary'].value
        xml_data['pension'] = user.attributes['pension'].value
        xml_data['company'] = user.attributes['company'].value
        xml_data['commute_distance'] = user.attributes['commute_distance'].value
        xml_data['address_postcode'] = user.attributes['address_postcode'].value
        xml_list.append(xml_data)
    
    return xml_list

In [ ]:
users_xml = read_xml_file('user_data.xml')

#### 1.3. Load the JSON Data

In [ ]:
def read_json(file):
    with open(file, 'r') as json_db:
        json_data = json.load(json_db)
    return json_data

In [ ]:
users_json = read_json('user_data.json')

In [ ]:
#printing datasets
from pprint import pprint
print(len(users_json))
pprint(users_json)


In [ ]:
print(len(users_xml))
pprint(users_xml)


In [ ]:
print(len(csv_db))
pprint(csv_db)


### 2. Merge the Datasets

In [ ]:
def merge(csv_data, json_data, xml_data):
    csv_json = [] #declaring an empty list for storing merged csv & json data
    #creating a nested for-loop to iterate through the dataset and join the csv & json data based on first name, lastname, age 
    for csv in csv_data: 
        for json in json_data: 
            if json['firstName'] == csv['First Name'] and json['lastName'] == csv['Second Name'] and json['age'] == int(csv['Age (Years)']):
                json['Sex'] = csv['Sex']
                json['Vehicle Make'] = csv['Vehicle Make']
                json['Vehicle Model'] = csv['Vehicle Model']
                json['Vehicle Type'] = csv['Vehicle Type']
                json['Vehicle Year'] = csv['Vehicle Year']
                csv_json.append(json)
    merged_data = [] #initializing an empty list for storing the merged data
    #creating a nested for loop for merging the csv_json data with the xml data based on first name, last name, sex & age
    for xml in xml_data:
        for cj in csv_json:
            if cj['firstName'] == xml['First Name'] and cj['lastName'] == xml['Last Name'] and cj['age'] == int(xml['age']) and cj['Sex'] == xml['sex']:
                cj['commute_distance'] = float(xml['commute_distance'])
                cj['company'] = xml['company']
                # due to inconsistencies in the dependants data we will use try-except to avoid errors
                try:
                    cj['dependants'] = int(xml['dependants'])
                except ValueError:
                    cj['dependants'] = 0
                cj['marital_status'] = xml['marital_status']
                cj['pension'] = float(xml['pension'])
                cj['retired'] = xml['retired']
                cj['salary'] = float(xml['salary'])
                merged_data.append(cj)      
    
    return merged_data

In [ ]:
users_merged = merge(csv_db, users_json, users_xml)

In [ ]:
print(len(users_merged))
pprint(users_merged)

### 3. Modify the Datasets based on Txt file

In [ ]:
class Change():
    """
    A class that changes the value of the dataset
    """
    def __init__(self, data_list):
        self.data = data_list
    
    #creating a function that iterates through the data and finds a first & last name that aligns and changes security code
    def change_sec_code(self, first_name, last_name, new_code):
        for user in self.data:
            if user['firstName'] == first_name and user['lastName'] == last_name:
                user['credit_card_security_code'] = new_code
                modified_data = user
        return modified_data
    
    #creating a function that increases the user salary based on first name & company
    def increase_salary(self, first_name, company, increase):
        for user in self.data:
            if user['firstName'] == first_name and user['company'] == company:
                user['salary'] = user['salary'] + increase
                modified_data = user
        return modified_data
    
    #creating a function that changes age based on first name and last name
    def change_age(self, first_name, last_name, age):
        for user in self.data:
            if user['firstName'] == first_name and user['lastName'] == last_name:
                user['age'] = age
                modified_data = user
        return modified_data
    
    #creating a function that changes pension based on last name and pension
    def change_pension(self, last_name, pension_value, percentage_increase ):
        for user in self.data:
            if user['lastName'] == last_name and user['pension'] == pension_value:
                user['pension'] = round((percentage_increase*0.01 +1) * user['pension'], 2)
                modified_data = user
        return modified_data
    

#### 3.1. Change Security code using firstname and last name

In [ ]:
#Creating an object of the class change to change the dataset based on different criteria
updated_list = Change(users_merged)

In [ ]:
updated_list.change_sec_code("Shane","Chambers", 935)

#### 3.2. Change salary using first name and company

In [ ]:
updated_list.increase_salary("Joshua", "Lewis-Johnson", 2100)

#### 3.3. Change age using surname and first name

In [ ]:
updated_list.change_age("Suzanne", "Wright", 37)

#### 3.4. Change pension using exiting pension and first name

In [ ]:
updated_list.change_pension("Dunn", 22358, 15)

### 4. Clean Data

In [ ]:
users_data = updated_list.data

Some inputs have debt while some don't and some of them are dict type while others are text. we will split the dict type to enable us store the different keys in the dict as a separate column in our db.

In [ ]:
def split_debt_input(dataset):
    for user in dataset:
        if 'debt' in user.keys():
            if(type(user['debt'])) is dict:
                user['debt_time_period_years'] = user['debt']['time_period_years']
                user['debt'] = user['debt']['amount']
    return dataset

In [ ]:
split_debt_input(users_data)

In [ ]:
len(users_data)

## B. SQL Database

In [ ]:
#importing pony orm & sqlite library
from pony.orm import *
import sqlite3

In [ ]:
#creating a new database and opening a connection to allow us work with it
sqlite_db = sqlite3.connect("User_db")

In [ ]:
#next we will create the database object

In [ ]:
db = Database()

In [ ]:
#creating the "User" entity and its attributes

In [ ]:
class User(db.Entity):
    first_name = Required(str)
    second_name = Required(str)
    age = Required(int)
    iban = Required(str)
    credit_card_number = Required(str)
    iban = Required(str)
    credit_card_number = Required(str)
    credit_card_security_code = Required(str)
    credit_card_start_date = Required(str)
    credit_card_end_date = Required(str)
    address_main = Required(str)
    address_city = Required(str)
    address_postcode = Required(str)
    debt = Optional(str)
    debt_time_period = Optional(int)
    Sex = Required(str)
    Vehicle_Make = Required(str)
    Vehicle_Model = Required(str)
    Vehicle_Type = Required(str)
    Vehicle_Year = Required(str)
    commute_distance = Required(float)
    company = Required(str)
    dependants = Required(int)
    marital_status = Required(str)
    pension = Required(float)
    retired = Required(str)
    salary = Required(float)

In [ ]:
#using the bind function to attach the User entity to the database

In [ ]:
db.bind('sqlite', 'C:/Users/HP OMEN/Documents/Programming/Data Science/Data Science Renewed/pythonassignment/Data/User_db')

In [ ]:
#creating the database tables for persisting our data using the generate mapping function

In [ ]:
db.generate_mapping(create_tables=True)

In [ ]:
#for printing the sql code while running our script
sql_debug(True)

In [ ]:
#creating entity instances/objects of our user entity

In [ ]:
#since we have little discrepancies in out data we will use a if-elif-else statement to ensure that all the data is parsed
# based on the differences in the debt column

In [ ]:
for user in users_data:
    if 'debt' and 'debt_time_period_years' in user.keys():
        user_db = User(
        first_name = user['firstName'],
        second_name = user['lastName'],
        age = user['age'],
        iban = user['iban'],
        credit_card_number = user['credit_card_number'],
        credit_card_security_code = str(user['credit_card_security_code']),
        credit_card_start_date = user['credit_card_start_date'],
        credit_card_end_date = user['credit_card_end_date'],
        address_main = user['address_main'],
        address_city = user['address_city'],
        address_postcode = user['address_postcode'],
        debt = user['debt'],
        debt_time_period = user['debt_time_period_years'],
        Sex = user['Sex'],
        Vehicle_Make = user['Vehicle Make'],
        Vehicle_Model = user['Vehicle Model'],
        Vehicle_Type = user['Vehicle Type'],
        Vehicle_Year = user['Vehicle Year'],
        commute_distance = user['commute_distance'],
        company = user['company'],
        dependants = user['dependants'],
        marital_status = user['marital_status'],
        pension = user['pension'],
        retired = user['retired'],
        salary = user['salary'],
        )
    elif 'debt' in user.keys():
        user_db = User(
        first_name = user['firstName'],
        second_name = user['lastName'],
        age = user['age'],
        iban = user['iban'],
        credit_card_number = user['credit_card_number'],
        credit_card_security_code = str(user['credit_card_security_code']),
        credit_card_start_date = user['credit_card_start_date'],
        credit_card_end_date = user['credit_card_end_date'],
        address_main = user['address_main'],
        address_city = user['address_city'],
        address_postcode = user['address_postcode'],
        debt = user['debt'],
        Sex = user['Sex'],
        Vehicle_Make = user['Vehicle Make'],
        Vehicle_Model = user['Vehicle Model'],
        Vehicle_Type = user['Vehicle Type'],
        Vehicle_Year = user['Vehicle Year'],
        commute_distance = user['commute_distance'],
        company = user['company'],
        dependants = user['dependants'],
        marital_status = user['marital_status'],
        pension = user['pension'],
        retired = user['retired'],
        salary = user['salary'],
        )
    else:
        user_db = User(
        first_name = user['firstName'],
        second_name = user['lastName'],
        age = user['age'],
        iban = user['iban'],
        credit_card_number = user['credit_card_number'],
        credit_card_security_code = str(user['credit_card_security_code']),
        credit_card_start_date = user['credit_card_start_date'],
        credit_card_end_date = user['credit_card_end_date'],
        address_main = user['address_main'],
        address_city = user['address_city'],
        address_postcode = user['address_postcode'],
        Sex = user['Sex'],
        Vehicle_Make = user['Vehicle Make'],
        Vehicle_Model = user['Vehicle Model'],
        Vehicle_Type = user['Vehicle Type'],
        Vehicle_Year = user['Vehicle Year'],
        commute_distance = user['commute_distance'],
        company = user['company'],
        dependants = user['dependants'],
        marital_status = user['marital_status'],
        pension = user['pension'],
        retired = user['retired'],
        salary = user['salary'],
        )
    commit()